In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'  # O '0,1' para usar múltiples GPUs
import tensorflow as tf
tf.test.gpu_device_name()

2024-09-05 21:19:07.989681: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 21:19:08.559455: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-05 21:19:09.191156: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 21:19:09.207959: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

'/device:GPU:0'

In [2]:
from PIL import Image
import numpy as np
from tensorflow.keras.utils import Sequence
from glob import glob

In [3]:
dataset = glob("dataset/*/*.jpg", recursive = True)

In [4]:
class Mygenerator(Sequence):
    def __init__(self, data, batch_size, readers, training = False):
        self.data = data
        self.batch_size = batch_size
        self.readers = readers
        self.training = training
    def __len__(self):
        return int(np.ceil(len(self.data)/float(self.batch_size)))
    
    def __getitem__(self,idx):
        batch = self.data[ idx * self.batch_size: (idx + 1) * self.batch_size]

        x = []
        y = []

        for filename in batch:
            x_, y_ = self.readers(filename,self.training)
            x.append(x_)
            y.append(y_)
        return np.array(x), np.array(y)

In [5]:
def wrapper_read_x_y(target_class):
    def read_x_y(filename, training = False):
        img = Image.open(filename).convert("RGB")
        img2 = img.resize((224,224))

        target = target_class in filename

        if not training:
            return np.array(img2) / 255,int(target)

        width, height = img2.size
        x_range, y_range = int(width * 0.3), int(height * 0.3)
        random_x = np.random.randint(0,x_range)
        random_y = np.random.randint(0,y_range)

        img3 = img2.crop((random_x, random_y, width - x_range, height - y_range)).resize((224,224))
        angle = np.random.randint(-5,5)
        img4 = img3.rotate(angle)

        return np.array(img4) / 255,int(target)
    return read_x_y

In [6]:
np.random.shuffle(dataset)

In [7]:
train = dataset[0:int(len(dataset)* .8)]
test = dataset[int(len(dataset)* .8):]

In [8]:
print(len(train))
print(len(test))

10000
2500


In [9]:
batch_size= 100
training_generator = Mygenerator(train,batch_size,wrapper_read_x_y("cats"),training = True)
test_generator = Mygenerator(test,batch_size,wrapper_read_x_y("cats"), training=False)

In [10]:
IMG_SHAPE = (224, 224, 3)
base_model = tf.keras.applications.mobilenet.MobileNet(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

2024-09-05 21:19:09.483352: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 21:19:09.483542: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-05 21:19:09.483644: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [11]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(1, activation='sigmoid')

In [12]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])


In [13]:
opt = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [14]:
model.fit(training_generator,validation_data=test_generator,batch_size=batch_size, epochs=1)


2024-09-05 21:19:13.064721: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-09-05 21:19:13.761117: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.49GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-09-05 21:19:13.771396: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x765383cf8790 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-05 21:19:13.771413: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2080 SUPER, Compute Capability 7.5
2024-09-05 21:19:13.810254: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2024-09-05 21:19:13.904113: W tensorflow/tsl/framew

100/100 [==============================] - 56s 506ms/step - loss: 0.0141 - accuracy: 0.9994 - val_loss: 0.0013 - val_accuracy: 1.0000


In [15]:
model.evaluate(test_generator, verbose=2)

25/25 - 9s - loss: 0.0013 - accuracy: 1.0000 - 9s/epoch - 356ms/step


[0.001280890661291778, 1.0]